# Avaliador sentimentos pelo CHAT GPT

- Autor: Daniel Saraiva Leite - 2023
- Projeto Análise de sentimentos sobre notícias do tema ESG
- Trabalho de conclusão de curso - MBA Digital Business USP Esalq

In [252]:
import openai
import os
import pandas as pd
import time
import re
import os
import hashlib

openai.api_key = os.getenv('GPT_API_KEY')


def classifica_sentimento_noticia_gpt2(texto, empresa, model='gpt-4-turbo-preview', caminho_cache='datasets/gpt_cache.xlsx', usar_cache=True, dicionario_cache=None):
    
    pergunta = "Você receberá um texto de uma notícia sobre uma empresa, e deverá responder: a) se a notícia do tema ESG; b) Em caso positivo, qual a dimensão dominante: E, S ou G; c) Avaliar o sentimento da notícia, considerando a atitude da empresa sob a ótica ESG, no intervalo real entre -1.0 até +1.0; d) Qual o nome da principal empresa envolvida; e) Esse texto tem como tema principal a empresa " + empresa + "? f) Se a notícia for do tema ESG, faça um breve resumo de apenas 1 sentença. Dê respostas curtas."
    
    
    texto_codificar = texto.strip() + empresa.strip() + pergunta.strip()
    
    hash_texto = hashlib.sha256(texto_codificar.encode('UTF-8')).hexdigest()
    
    caminho_cache='datasets/gpt_cache.xlsx'
    
    if usar_cache:
        if dicionario_cache is not None:
            dic= dicionario_cache
        else:
            df = pd.read_excel(caminho_cache)
            dic = df.set_index('hash')['resposta'].to_dict()


    if usar_cache and hash_texto in dic:
        return dic[hash_texto]
    else:
        response = openai.ChatCompletion.create(
                    model=model,
                    messages=[
                        {
                          "role": "system",
                          "content": pergunta
                        },
                        {
                          "role": "user",
                          "content": texto
                        }
                      ],
                    temperature=0,
                    max_tokens=200,
                    top_p=1,
                    frequency_penalty=0,
                    presence_penalty=0
                    )


        resposta = response.choices[0].message["content"]
        dic[hash_texto] = resposta
        if dicionario_cache is  None: # salva cache caso nao seja controlado pelo chamador
            df = pd.DataFrame({'hash': dic.keys(),  'resposta': dic.values()})
            df.to_excel(caminho_cache, index=False)

    return resposta







def trata_resposta_gpt(r):
    
    itens = ['a) ', 'b) ', 'c) ', 'd) ', 'e) ', 'f) ', 'g) ']
    
    r = r.replace('\n\n', '\n')
    
    respostas = r.split('\n')
    
    
    respostas = [resp.replace(item, '') for resp, item in zip(respostas, itens)]
    
    tema = respostas[0][:3]
    
    if tema == 'Sim':
        tema = 'ESG'
    else:
        tema = 'Outros'
    
    dimensao = 'Outros'
    if 'S' in respostas[1] or 'G' in respostas[1] or 'E' in respostas[1]:
        dimensao = respostas[1].replace('A dimensão dominante é ', '').replace('"', '').strip()[:1]
        
    sentimento = 0
    nota = re.findall('[+-]?[0-9][.]?[0-9]{1,3}', respostas[2])
    if len(nota) > 0:
        sentimento = float(nota[0].strip())
        
    empresa_detectada = respostas[3].replace('A principal empresa envolvida é o ', '').replace('A principal empresa envolvida é a ', '').replace('.', '')
    
    #cnpj = ''
    #cnpj_texto = re.findall('[[0-9]{1,3}[.][0-9]{1,3}[.][0-9]{1,3}[/][0-9]{4,4}-[0-9]{2}', respostas[4])
    #if len(cnpj_texto) > 0:
    #    cnpj = cnpj_texto[0]
        
    foco_empresa = respostas[4][:3]
    
    resumo = respostas[5]
    
    return [tema, dimensao, sentimento, empresa_detectada, foco_empresa, resumo]


def gera_colunas_gpt(df, coluna_resposta_gpt='resposta'):
    df['gpt_lista_respostas'] = df.apply(lambda row: trata_resposta_gpt(row[coluna_resposta_gpt]), axis=1)
    df['gpt_tema_esg'] = df[''].apply(lambda x : x[0])
    df['gpt_classificacao'] = df['gpt_lista_respostas'].apply(lambda x : x[1])
    df['gpt_polaridade'] = df['gpt_lista_respostas'].apply(lambda x : x[2])
    df['gpt_empresa_principal'] = df['gpt_lista_respostas'].apply(lambda x : x[3])
    df['gpt_focado_empresa'] = df['gpt_lista_respostas'].apply(lambda x : x[4])
    df['gpt_resumo'] = df['gpt_lista_respostas'].apply(lambda x : x[5])


In [217]:
resp = classifica_sentimento_noticia_gpt(t, empresa='braskem', usar_cache=False)
print(resp)
trata_resposta_gpt(resp)

a) Sim, a notícia é do tema ESG.

b) E (Ambiental).

c) -0.8 (A notícia tem um sentimento bastante negativo, dada a multa por danos ambientais e a exclusão de um índice de sustentabilidade).

d) Braskem.

e) Sim, o texto tem como tema principal a empresa Braskem.

f) A Braskem foi multada em mais de R$ 72 milhões pelo IMA-AL por danos ambientais e omissão de informações, levando à sua exclusão do Índice de Sustentabilidade Empresarial da B3.


['ESG',
 'Outros',
 -0.8,
 'Braskem',
 'Sim',
 'A Braskem foi multada em mais de R$ 72 milhões pelo IMA-AL por danos ambientais e omissão de informações, levando à sua exclusão do Índice de Sustentabilidade Empresarial da B3.']

In [196]:
'a) Sim, a notícia é do tema ESG.\nb) A dimensão dominante é E (ambiental).\nc) O sentimento da notícia é -1.0, pois apresenta informações negativas sobre a empresa.\nd) A principal empresa envolvida é a Braskem.\ne) O CNPJ da Braskem é 42.150.391/0001-70.\nf) Sim, o texto tem como tema principal a empresa Braskem.'

'a) Sim, a notícia é do tema ESG.\nb) A dimensão dominante é E (ambiental).\nc) O sentimento da notícia é -1.0, pois apresenta informações negativas sobre a empresa.\nd) A principal empresa envolvida é a Braskem.\ne) O CNPJ da Braskem é 42.150.391/0001-70.\nf) Sim, o texto tem como tema principal a empresa Braskem.'

In [3]:
t = '''
Alagoas multa Braskem em mais de R$ 72 milhões por danos ambientais
Desde 2018 a empresa foi autuada 20 vezes pelo IMA

Redação
Alagoas multa Braskem em mais de R$ 72 milhões por danos ambientais
O Instituto do Meio Ambiente do Estado de Alagoas (IMA-AL) autuou a Braskem em mais de R$ 72 milhões por omissão de informações, danos ambientais e pelo risco de colapso e desabamento da mina 18, na região do Mutange, em Maceió. Conforme divulgou o IMA nesta terça-feira (5), desde 2018 a Braskem já foi autuada 20 vezes pelo instituto.

O IMA informou que a primeira multa, no valor de R$ 70.274.316,34, foi feita devido à degradação ambiental decorrente de atividades que, direta ou indiretamente, afetam a segurança e o bem-estar da população, “gerando condições desfavoráveis para as atividades sociais e econômicas”.

Um estudo do IMA já havia constatado dano ambiental na região da mina 18. A nova ocorrência de colapso da mina, verificada in loco, foi caracterizada como reincidência.

Braskem

“Além dessa autuação, a Braskem vai responder também pela omissão de informações sobre a obstrução da cavidade da mina 18, detectada no dia 07/11/2023, quando a empresa realizou o exame de sonar prévio para o início do seu preenchimento, em desconformidade com a Licença de Operação n° 2023.18011352030.Exp.Lon. A multa é de R$ 2.027.143,92”, informou o instituto.

B3 exclui Braskem do Índice de Sustentabilidade Empresarial

A B3 informou nesta terça-feira, 5, que as ações da Braskem deixarão de integrar a carteira de seu Índice de Sustentabilidade Empresarial (ISE B3) a partir da próxima sexta-feira, 8, à medida que a Prefeitura de Maceió decretou situação de emergência envolvendo uma mina da petroquímica. O índice reúne as empresas com melhores práticas de sustentabilidade ambiental, social e de governança listadas na bolsa brasileira

A decisão tomada pela B3 considerou os quatro pilares divulgados no Plano de Resposta a Eventos ESG, instaurado pela companhia na última sexta-feira, 1º: o impacto ESG da crise, gestão da crise pela companhia, impacto de imagem da crise na companhia; e resposta da companhia à crise.
'''

In [172]:
t2 = '''
Parceira de hotéis da Hard Rock no Brasil é investigada e quer captar R$ 607 milhões
Companhia que toca projeto com atraso de 5 anos coleciona reclamações e promete entregar obra até o fim de 2023

Estadão Conteúdo
Há quase dez anos, um grupo de cotistas aguarda para exercer seu direito de relaxar em um resort de luxo que está sendo erguido em Sertaneja, município com menos de 10 mil habitantes na região Norte do Paraná (a cerca de 80 km de Londrina). A entrega, prevista inicialmente para 2013, agora está prometida para o ano que vem. Desde então, o projeto já mudou de mãos e ganhou uma nova marca poderosa – Hard Rock Resorts -, mas mesmo assim ainda não saiu do papel.

O projeto hoje está na mão da Venture Capital Participações e Investimentos (VCI), parceira do grupo americano no Brasil, que anunciou o acordo em dezembro de 2017, assumindo dois empreendimentos do tipo – o do Paraná e um no Ceará – cujos donos originais tinham enfrentado dificuldades financeiras e que ainda continuam na promessa.

Hoje, quase 6 mil cotistas dividem o direito de propriedade do resort no norte paranaense: entre quem fez o investimento, reina a incerteza sobre quando o hotel ficará pronto e certo desânimo com a aquisição. Desde o início da parceria com a Hard Rock, o VCI lançou empreendimentos com potencial de gerar R$ 8 bilhões em vendas.

Continua depois da publicidade

A companhia segue vendendo cotas dos hotéis em regime de frações (multipropriedade) e montou uma nova operação no mercado de capitais que pretende arrecadar um total de R$ 607 milhões para financiar as obras. A primeira fase da captação, de R$ 230 milhões, já está em andamento.

Apesar dos recursos levantados e do fluxo contínuo de vendas, os clientes se queixam há meses do ritmo lento de evolução das obras e da falta de prestação de contas. “Tem sempre gente batendo martelo lá, quebrando uma parede aqui ou ali, mas do jeito que está indo vão mais de 20 anos para acabar a obra”, diz um cliente que costuma visitar o local no Paraná e esteve na obra há cerca de dois meses.

INVESTIGAÇÕES

Mas o ceticismo dos clientes não se resume a reclamações em redes sociais e em órgãos de direito do consumidor. Vai muito além. Suspeitas de irregularidades recaem sobre os responsáveis pelos projetos em dois processos na Comissão de Valores Mobiliários (CVM). Procurado, o órgão disse não comentar processos em andamento. As suspeitas levantadas pelo regulador do mercado são averiguadas pela Polícia Federal e pelo Ministério Público Federal.

Procurada, a empresa nega irregularidades e afirma que até o momento nada foi comprovado em nenhuma instância, mesmo após anos de verificações. Também sustenta que parte das acusações se refere a captações de recursos feitas por um antigo dono de um dos projetos. A companhia ainda disse ter passado por diligências de cinco meses e afirma acreditar em sua absolvição no julgamento da CVM, aguardado para dezembro.

Procurado, o Hard Rock Internacional afirmou que monitora todas as fases dos seus negócios no País. “O Brasil é e continuará a ser um mercado importante para a Hard Rock International”, afirmou a empresa, em nota. A rede americana disse também que será responsável pela operação e pela gestão dos hotéis do Paraná e de Fortaleza. “A VCI é a parceira para a incorporação, a construção e a comercialização dos empreendimentos, no modelo multipropriedade.”

Obras evoluem lentamente, empresa tenta acelerar

Com centenas de pedidos de rescisão de contratos na mesa por causa de atrasos em obras, o fundo VCI corre para dar ritmo a suas obras. No Paraná, segundo a empresa, o número de trabalhadores em atividade saltou de 114 para 240, com a expectativa de alcançar mais de 600 no início de 2023. No Ceará, são hoje 608 trabalhadores, com expectativa de atingir mais de mil no ano que vem.

A evolução é acompanhada por auditorias da empresa Cushman & Wakefield, especializada na área imobiliária. A VCI também trouxe profissionais com experiência no mercado imobiliário, como o CEO Fábio Villas-Bôas, ex-Tecnisa.

Documentos sobre a nova captação de debêntures informam que, em agosto, as obras estavam com evolução de 33,8% no Paraná e 53,5% no Ceará.

A empresa alega que precisou fazer adaptações ao padrão da marca internacional, inclusive com a demolição de partes existentes e alega que 46,44% da obra está concluída no Paraná e 65,17%, no Ceará.

As informações são do jornal O Estado de S. Paulo.
'''

In [173]:
classifica_sentimento_noticia_gpt(t2, 'vci')

'a) Sim\n\nb) G\n\nc) -0.6\n\nd) Venture Capital Participações e Investimentos (VCI)\n\ne) Não disponível\n\nf) Sim'

In [201]:
t3 = '''

TOP 10 celulares mais poderosos do mundo, segundo o AnTuTu
Qual o melhor celular do mundo? Confira agora a lista completa dos modelos mais poderosos do mundo, segundo os testes de benchmark do AnTuTu!

Adalton Bonaventura07/02/2024 11:55
O AnTuTu finalmente revelou a lista dos celulares mais poderosos de janeiro de 2024. Temos um novo líder neste mês: o Red Magic 9 Pro, celular gamer da Nubia lançado no final do ano passado com o novo processador Snapdragon 8 Gen 3 e incríveis 16 GB de memória RAM. Foi por pouco, mas ele superou o iQOO 12, líder do mês passado e agora é o primeiro colocado. Destaque para a Samsung cinco modelos diferentes na lista.

TOP 10 celulares mais poderosos no AnTuTu

Qual o melhor celular do mundo? Essa lista é baseada nos modelos globais e desenvolvida pelo AnTuTu.

O que é o AnTuTu?

O AnTuTu benchmark é um aplicativo para Android e iOS que, ao instalar em seu celular, você pode rodar um teste que pontua ações que ele analisa. O teste inclui ações simples como ler um QRCode, navegar na internet e redes sociais até renderizar vídeos, fazendo uma varredura completa, simulando assim o uso cotidiano das pessoas. Ao final deste teste, sai uma pontuação geral, e é essa pontuação que o aplicativo usa como parâmetro para determinar os celulares mais potentes do mundo.
Qual o melhor celular do mundo?

Abaixo, esses são os cinco melhores celulares do mundo segundo o AnTuTu:

1. Red Magic 9 Pro

 Red Magic 9 Pro
Red Magic 9 Pro
O Red Magic 9 Pro é o novo líder do ranking do AnTuTu, batendo o título de celular mais poderoso do mundo no mês de janeiro de 2024. Para isso, o celular gamer da Nubia vem com o processador Snapdragon 8 Gen 3, 16 GB de RAM e 512 GB de armazenamento. Sua pontuação média foi de quase 2,1 milhões de pontos.

2. iQOO 12

 iQOO 12
iQOO 12
O novíssimo iQOO 12 chegou ao mercado há quase três meses e com uma ficha técnica de dar inveja. Debaixo do capô, ele carrega o novo processador Snapdragon 8 Gen 3, e tem opções com até 16 GB de RAM e 512 GB de armazenamento. Com esse pacote de especificações, o aparelho alcançou uma média total de 2 milhões de pontos no AnTuTu.

3. Nubia Z60 Ultra

 Nubia Z60 Ultra
Nubia Z60 Ultra
Lançado no final do ano passado, o Nubia Z60 Ultra é um celular poderoso, equipado com o processador Snapdragon 8 Gen 3, 16 GB de memória RAM e 512 GB armazenamento. Embora seja o terceiro colocado, sua pontuação não ficou muito longe do primeiro lugar. Durante o mês de janeiro, sua pontuação média também superou os 2 milhões de pontos.

4. vivo X100 Pro

 vivo X100 Pro
vivo X100 Pro
O vivo X100 Pro é o único dessa lista com um processador MediaTek, no caso o novo Dimensity 9300. Além disso, ele vem equipado com 16 GB de RAM, 512 GB de armazenamento. Esse pacote de especificações garantiram a ele a quarta colocação do mês de janeiro com uma pontuação média de 1.9 milhão de pontos.

5. Galaxy S24 Ultra

 Galaxy S24 Ultra
Galaxy S24 Ultra
O Galaxy S24 Ultra é o quinto colocado do ranking e o primeiro dos cinco modelos da Samsung a aparecer no top 10. Ele passou pelos testes de benchmark do AnTuTu e ficou com uma média de 1.8 milhão de pontos neste mês de janeiro. Para isso, ele chega ao mercado equipado com o Snapdragon 8 Gen 3, 12 GB de RAM e 1 TB de armazenamento.

Exynos surpreende em janeiro

Todos os meses que atualizamos esse ranking, a Qualcomm sempre se sobresai por causa do número de celulares que utilizam os processador da marca. Em janeiro de 2024 não é diferente, afinal, sete dos dez modelos dessa lista usam algum modelo de processador Snapdragon.

A surpresa fica mesmo para os chips Exynos, fabricados pela Samsung. Tanto o Galaxy S24+ e S24 "padrão" conseguiram superar modelos equipados com o Snapdragon 8 Gen 2, mostrando que a Samsung tem feito um ótimo traballho no quesito desempenho dos seus chips. A MediaTek aparece apenas no quarto lugar com o vivo X100 Pro equipado com o Dimensity 9300.

Qual a marca do seu smartphone?

Qual a marca do seu smartphone? Queremos saber, vote abaixo:

Gostou do ranking? Aqui no Oficina da Net todos os celulares que testamos passam por várias baterias de testes que também criamos os nossos próprios rankings. Dá uma conferida aí:

Fonte: AnTuTu
'''

In [202]:
classifica_sentimento_noticia_gpt(t3, empresa='Vivo')

'a) Não, a notícia não é do tema ESG.\nb) Não se aplica.\nc) 0.1 (A notícia é neutra, mas ligeiramente positiva devido ao tom informativo sobre avanços tecnológicos).\nd) Nubia (Red Magic 9 Pro é mencionado como o líder do ranking).\ne) Não, o tema principal não é a empresa Vivo, embora seja mencionada.\nf) Não se aplica.'

In [208]:
t4 = '''


Bradesco pagará dividendos atrativos após reestruturação?
Bruno Andrade
As ações do () recuavam no pregão desta sexta-feira (9) após o banco ter reportado um de R$ 35,6 bilhões em 2023, alta de 11,4% na comparação com o mesmo período do ano passado.

Leia também

Por volta de 14h50, as ações do Banco do Brasil recuavam 1,74%, a R$ 57,52. Para Gabriel Meira especialista e sócio da Valor Investimentos, a queda de hoje é apenas uma realização de lucro do mercado, visto que o lucro de R$ 9,44 bilhões no quarto trimestre de 2023 ficou acima dos R$ 9,2 bilhões esperados pelos analistas. “O consenso era de um bom lucro e o número foi bem melhor: a queda de hoje é apenas uma realização de lucro do mercado, que faz as ações subirem no boato e caírem no fato”, avalia.

Já Milton Rabelo, analista da VG Research, diz acreditar que a queda pode estar ocorrendo após o banco projetar um crescimento entre 12,5% e 3,9% do lucro em 2024. Segundo ele, isso pode sinalizar uma desaceleração no crescimento do lucro da instituição em 2024, visto que em 2023 o crescimento foi de 11,4%.

Ainda que a ação do BB esteja em queda nesta sexta-feira, as casas de analises recomendam compra. Segundo os analistas da XP Investimentos, o banco apresentou “resultados impressionantes” principalmente após a companhia ter a maior rentabilidade, medida pelo Retorno sobre o Patrimônio Líquido (ROE), entre os quatros grandes bancos no quarto trimestre de 2023. O indicador do BB encerrou 2023 em 22,5%.

Dividendos do Banco do Brasil

Outro fator que agradou os analistas foi a questão dos dividendos. A empresa anunciou a elevação do porcentual do lucro pago em proventos (payout) de 40% para 45%.Com base nas projeções de lucro e no payout estimado pela empresa, o Banco do Brasil pode pagar entre R$ 16,6 bilhões e R$ 18 bilhões em proventos neste ano.

Os analistas da Ativa Investimentos viram esse movimento de aumento do payout como positivo. “Consideramos esse movimento bastante positivo e similar ao forte guidance de lucro divulgado, onde a diretoria do BB dá um importante recado em relação ao seu alinhamento de interesse com os acionistas, o que também deverá ser bem recebido pelo mercado”, afirma Pedro Serra em relatório enviado ao mercado nesta sexta-feira (9).

Banco do Brasil pagará mais dividendos com lucro recorde: hora de comprar?
Os analistas estão otimistas e recomendam compra para BB. A XP Investimentos acredita que mesmo que o papel tenha subido 68,5% entre 29 de dezembro de 2022 e a última quinta-feira (8), o ativo ainda está descontado.

Por isso, a recomendação é de compra com preço-alvo de R$ 61, alta de 4,2% na comparação com o fechamento de quinta. Enquanto a Genial acredita que o papel deve subir 10,9% e chegar aos R$ 64,90, a Ativa Investimentos é mais otimista e acredita que o investidor deve entender que a divulgação dos resultados de hoje diminuíram o risco estatal.

“O temor do risco estatal de fato nos parece que foi sendo diluído ao longo de 2023 e enxergamos algumas razões, com destaque para o período em que o governo quis reduzir o teto de juros do INSS para 1,7% ao mês, patamar que não era rentável para o setor bancário. Os bancos públicos, em linha com os privados, se recusaram a reduzir os juros nesse patamar”, explicam os analistas da Ativa. A corretora possui recomendação de compra para o papal do Banco do Brasil com preço-alvo de R$ 75, uma alta de 28,1% na comparação com o fechamento de quinta-feira (8)

A Empiricus Research prefere o Itaú por causa da boa entrega do banco e pelo questão da instituição financeira ser privada e não ter o risco estatal. A corretora não tem preço-alvo para o papel.

A visão do CFO do Banco do Brasil

O E-Investidor questionou os executivos do Banco do Brasil, em entrevista coletiva nesta sexta-feira (9), se a instituição poderia ir além do pagamento de 45% de pagamento de dividendos como piso, remunerando os investidores com um valor a mais dependendo do desempenho do banco ao longo de 2024.

Geovanne Tobias CFO Banco do Brasil
Geovanne Tobias, CFO Banco do Brasil. (Foto: Banco do Brasil/Divulgação)
Essa política é feita pelo Itaú, que tem como um piso de payout em 30%, mas pagou 60% do lucro em dividendos. Todavia, o CFO do Banco Brasil, Geovanne Tobias, negou essa possibilidade. “Preferimos uma estabilidade com o pagamento de 45% do lucro em dividendos, do que um payout baixo com dividendos extraordinários como o do Itaú”.

Ele comentou que prefere ter esse payout fixo mais elevado, pois “ele traz mais previsibilidade e segurança para o investidor. O acionista vai saber ao longo do ano o porcentual do lucro que ele vai receber”.

Os executivos do Banco do Brasil disseram que os investidores não devem se preocupar com a possibilidade de intervenção estatal na empresa, principalmente após o primeiro ano da gestão de Tarciana Medeiros.

Bradesco promove reestruturação. Banco agora pagará dividendos atrativos?
Segundo Tobias, a mensagem que pode ficar para o investidor é que todos os funcionários do banco são de carreira e conhecem o DNA da companhia. “Nós sabemos o que estamos fazendo, apesar de todo descrédito que o mercado deu para o Banco do Brasil e para esta gestão”, afirmou Tobias, lembrando que a companhia encerrou 2023 com o maior lucro de sua história e com o maior ROE entre os quatro bancos.

Tobias também ironizou quem segue com receio de o governo interferir na estratégia do BB. “Tem gente que acredita em Papai Noel, tem gente que acredita que a Terra é plana. O que que eu posso fazer?”.

Nossos editores indicam estes conteúdos para você investir cada vez melhor

'''

In [212]:
r = classifica_sentimento_noticia_gpt(t4 , empresa='BB')

In [213]:
r

'a) Sim, a notícia é do tema ESG.\nb) G (Governança).\nc) +0.5 (A notícia tem um tom positivo em relação ao desempenho financeiro do Banco do Brasil e suas políticas de dividendos, apesar de mencionar a queda das ações).\nd) Banco do Brasil.\ne) Sim, o tema principal é a empresa BB.\nf) O Banco do Brasil anunciou um aumento no pagamento de dividendos após reportar um lucro recorde, refletindo uma governança corporativa alinhada aos interesses dos acionistas.'

In [94]:


    
trata_resposta_gpt(r)

('ESG', 'E', -0.8, 'Braskem', '42.150.391/0001-70', 'Não')

In [88]:
r

'a) Sim, a notícia é do tema ESG.\nb) A dimensão dominante é G (Governança).\nc) O sentimento da notícia é negativo, avaliado em -0.8.\nd) A principal empresa envolvida é a Venture Capital Participações e Investimentos (VCI).\ne) O CNPJ da principal empresa envolvida não é mencionado na notícia.\nf) Não, o texto não tem como tema principal a empresa BB.\ng) A Venture Capital Participações e Investimentos (VCI), parceira da Hard Rock no Brasil, está sendo investigada por atrasos e reclamações em um projeto de resort de luxo no Paraná. A empresa, que assumiu o projeto em 2017, promete entregar a obra até o fim de 2023. Apesar de continuar vendendo cotas do hotel e planejar arrecadar R$ 607 milhões para financiar as obras, os clientes reclamam do ritmo lento de construção e falta de prestação de contas. A VCI também enfrenta investigações da Comissão de Valores Mobiliários (CVM) por suspeitas de irregularidades.'

In [246]:
df = pd.read_excel('datasets/gpt_cache_testes.xlsx')


In [249]:
df.to_excel('datasets/gpt_cache_testes.xlsx', index=False)

In [255]:
t = '''
Signatário da Carta em defesa do Estado Democrático de Direito da USP, Pedro Luiz Passos, um dos fundadores da Natura, diz que a classe empresarial acordou para um risco de retrocesso institucional nestas eleições, com os questionamentos do presidente Jair Bolsonaro à segurança da urna eletrônica, e associa o engajamento a razões, inclusive, pragmáticas:

— A ruptura institucional no país penalizaria também o ambiente de negócios.

Qual a importância de assinar um manifesto pela democracia neste momento?

Acho que toda a população brasileira, a que está informada pelo menos, tem medo e vê risco de um retrocesso institucional. Para todos os segmentos: o cidadão comum, o empresarial, a cultura, as artes... A manifestação da sociedade civil neste momento é muito importante para evitar o retrocesso. Até então, essas manifestações não estavam coordenadas, mas dessa vez a adesão foi muito rápida.

Como interpretar a velocidade dessa adesão? Ela seria proporcional a uma percepção de risco de uma ruptura?

É um alerta da sociedade que não quer esse caminho. Não acredito na efetividade de um golpe com tanques na rua, como no passado. Vejo uma ruptura possível das instituições se a gente não respeitar o direito de voto dos cidadãos. O questionamento à urna eletrônica é absurdo e pode fragilizar a nossa democracia, fazer a gente retroceder.

Ao longo de toda a atual gestão, a sociedade civil divulgou uma série de notas de repúdio e manifestos. Mas esses movimentos não contavam com a adesão da classe empresarial, com algumas exceções. Isso mudou?

Vejo um movimento crescente de cidadania empresarial. Se não por outro motivo, há uma razão pragmática. Uma ruptura institucional no país penalizaria também o ambiente de negócios. É um momento difícil do país, mas é bonito ver diversos setores se manifestando suprapartidariamente, sem linha ideológica de esquerda ou de direita. Movimento que está falando que quer o caminho da democracia. Nossas lideranças empresariais mais históricas ficam um pouco distantes de manifestações por medo de represália ou de conflitos de interesse com acionistas. Mas hoje está evidente que também precisamos do empresário para corrigir o rumo de desrespeito às regras do jogo.

O senhor está engajado na candidatura da Simone Tebet. Apesar de contar com o apoio de lideranças empresariais, intelectuais e economistas, ela não sai dos 2%. O que falta para ela chegar na base da pirâmide?

Ela vai chegar nos dois dígitos, não tenho dúvida. Sinto que na medida em que a Simone vai ficando mais exposta, pessoas que não a conheciam passam a considerá-la para o primeiro turno. Acho que no primeiro turno a gente deveria aproveitar para expor ideias, discutir programa; isso a campanha da Simone traz, com gente jovem, moderna, que pensa no futuro do país. Acho que as pessoas deveriam usar o primeiro turno para escolher a opção que consideram a melhor. Aí no segundo você vota contra quem você não quer de jeito nenhum.

Paulo Guedes foi fundamental para atrair o empresariado para a candidatura de Jair Bolsonaro na eleição passada. Qual a sua avaliação da gestão do ministro da Economia?

A avaliação é não só do Paulo Guedes, mas da gestão como um todo: não entregou o que sinalizava nas eleições. Existem várias formas de estelionato eleitoral e essa é uma delas. Porque tinha uma agenda supostamente liberal que não foi liberal. Acho que é um pecado combater o liberalismo dizendo que Bolsonaro é liberal. Isso é tudo que ele não é. Ele é conservador populista. O Paulo Guedes, não sei se por razões internas, políticas, do próprio governo, não entregou o que prometeu. Temos que resolver o problema da desigualdade, saúde e educação. Mas isso não quer dizer que a gente não tenha que ter agenda moderna.

E o que seria uma agenda moderna?

Contempla uma preocupação social, o endereçamento da fome. Mas você também simplifica processos, faz uma alocação de recursos correta. E o Brasil tem muitos puxadinhos que tiram até a flexibilidade orçamentária… Porque temos 80% ou 90% do orçamento público que são despesas obrigatórias. O Brasil precisa se modernizar e se inserir no mundo. Fazer parte das grandes discussões ambientais, econômicas. Fazer acordos multilaterais, bilaterais. Tem que ter presença, ser voz a ser ouvido. O Brasil tem muito a entregar na pauta da agenda climática. Eu diria que a decepção é geral com esse governo.

O que explica uma parte expressiva do empresariado ainda apoiar o governo apesar desse diagnóstico de não ter entregue uma gestão liberal?

Em primeiro lugar, ele facilitou a vida do agronegócio. Eu falo do pequeno agronegócio. Porque as grandes empresas do setor agem dentro das regras do jogo. Mas a vida dos pequenos que exploram invasão, que exploram garimpo, ele facilitou, tirando recursos dos órgãos de controle, sobretudo na parte ambiental. Isso criou um falso liberalismo que atendeu algumas necessidades do pequeno e médio empresário. Então acho que hoje o apoio vem mais dos pequenos e médios do que das empresas mais estruturadas.

E para esse segmento, a pauta da liberação das armas é muito sensível.

Uma imprudência, para dizer o mínimo, facilitar a compra, despejar o número de armas que ele despejou e ter um povo armado, com a carência que se tem, com a presença do crime organizado. Um malfeito enorme para o país.

O que o senhor teme em uma eventual volta de Lula ao poder?

O PT está propondo uma agenda antiga para um mundo que mudou. E o país está em uma situação ainda mais grave. Fala-se na volta da intervenção do Estado. Quererem recompor a estatização da Petrobras, entrando em distribuição, refinarias. Já disse que não sou favorável ao Estado mínimo. Defendo um Estado que entrega serviço para a população. Mas o Estado operador, que Dilma (Rousseff) tanto gosta, que o Lula e o PT gostam, pode ser um tiro no pé. Imagina ser contra a lei de saneamento. Tem gente do PT que é contra. O setor público não entregou nada nessa área, por que manter a mesma direção? Estamos pagando a conta até hoje. A agenda do passado vai afundar o país.

Esse discurso mais radical é para ganhar eleição e, no poder, o Lula repetiria o primeiro mandato ou o senhor acha que ele deve entregar o que está prometendo na campanha?
'''


classifica_sentimento_noticia_gpt(t, 'natura', usar_cache=False)

'a) Sim, a notícia é do tema ESG.\nb) G (Governança).\nc) 0.0 (neutro).\nd) Natura.\ne) Sim.\nf) Pedro Luiz Passos, cofundador da Natura, destaca a importância do engajamento empresarial na defesa da democracia e do Estado de Direito no Brasil, em face dos riscos de retrocesso institucional.'